In [29]:
import datetime
import os
import pathlib
from typing import Union

import mlflow
import numpy as np
import yaml
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
cwd = pathlib.Path(os.getcwd())
cwd.absolute()

PosixPath('/Users/marek.zalecki/Projects/ml-serco/src/notebooks')

In [10]:
def read_yaml_config(config_filepath: Union[str, pathlib.Path]):
    with open(config_filepath) as yaml_file:
        config_ = yaml.safe_load(yaml_file)
    return config_


config_filepath = '../../config/mlflow.yaml'
config = read_yaml_config(config_filepath)

In [23]:
dirpath = pathlib.Path(f"../../{config['mlflow']['dirpath']}").resolve()
mlflow.set_tracking_uri(f"file://{dirpath}")
mlflow.autolog()

2023/04/27 15:00:48 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [34]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [35]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1600)             

In [36]:
def register_experiment(experiment_name: str):
    experiment_ = mlflow.get_experiment_by_name(experiment_name)
    if experiment_ is None:
        _ = mlflow.create_experiment(name=experiment_name)
        experiment_ = mlflow.get_experiment_by_name(experiment_name)

    return experiment_


experiment = register_experiment('notebooks')

In [37]:
with mlflow.start_run(
        run_name=datetime.datetime.now().isoformat(),
        experiment_id=experiment.experiment_id,
        tags={"version": "v1", "priority": "P1"},
        description="mnist-test-runtime",
):

    batch_size = 32
    epochs = 15
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

Epoch 1/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2226 - accuracy: 0.9321 - val_loss: 0.0621 - val_accuracy: 0.9825
Epoch 2/15
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0861 - accuracy: 0.9734 - val_loss: 0.0403 - val_accuracy: 0.9893
Epoch 3/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0662 - accuracy: 0.9801 - val_loss: 0.0386 - val_accuracy: 0.9888
Epoch 4/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0568 - accuracy: 0.9817 - val_loss: 0.0354 - val_accuracy: 0.9915
Epoch 5/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0513 - accuracy: 0.9837 - val_loss: 0.0321 - val_accuracy: 0.9910
Epoch 6/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0449 - accuracy: 0.9860 - val_loss: 0.0334 - val_accuracy: 0.9912
Epoch 7/15
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0420 - accuracy: 0.9867 - val_loss: 0.0308 - val_acc

INFO:tensorflow:Assets written to: /var/folders/tv/6g181qv92bn3kk8d9fkwpq3h0000gn/T/tmpjlc2xmi9/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/tv/6g181qv92bn3kk8d9fkwpq3h0000gn/T/tmpjlc2xmi9/model/data/model/assets


Test loss: 0.022125694900751114
Test accuracy: 0.9936000108718872
